In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from pickle import dump
import gc
import time

In [2]:
start_time = time.time()

In [3]:
pairs = pd.read_pickle('../input/fourpoints-location-matching/train_pairs.pkl')

In [4]:
#pairs.head()

In [5]:
#pairs.columns

In [6]:
cols = ['latitude_1','longitude_1','country_1','latitude_2','longitude_2','country_2','name_1_fuzzy', 'address_1_fuzzy', 'categories_1_fuzzy','url_1','url_2','phone_1','phone_2']
ids = ['id_1','id_2']

In [7]:
train_id = pairs[ids]
train_X = pairs[cols]
train_X['country_same'] = np.where(train_X['country_1'].astype(object) == train_X['country_2'].astype(object),1,0)
del train_X['country_1'] 
del train_X['country_2']
gc.collect()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


23

In [8]:
col_scaler = ['latitude_1','longitude_1','latitude_2','longitude_2']
X_transform = train_X[col_scaler]
scaler = RobustScaler().fit(X_transform)
dump(scaler, open('scaler.pkl', 'wb')) # save the scaler
train_X[col_scaler] = scaler.transform(X_transform)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [9]:
train_y = pairs['match']
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
              importance_type='gain',
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=2, n_estimators=400, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.9, tree_method='exact', validate_parameters=1,
              verbosity=None)
xgb.fit(train_X, train_y)
xgb.save_model("model.json")

In [10]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 1051.7016909122467 seconds ---


# download model
<a href="./scaler.pkl"> scaler </a>

<a href="./model.json"> xgboost model </a>

In [11]:
#pairs.dtypes.apply(lambda x: x.name).to_dict()

{'id_1': 'category',
 'name_1': 'object',
 'latitude_1': 'float16',
 'longitude_1': 'float16',
 'address_1': 'object',
 'city_1': 'category',
 'state_1': 'category',
 'zip_1': 'category',
 'country_1': 'category',
 'url_1': 'int8',
 'phone_1': 'int8',
 'categories_1': 'object',
 'id_2': 'category',
 'name_2': 'object',
 'latitude_2': 'float16',
 'longitude_2': 'float16',
 'address_2': 'object',
 'city_2': 'category',
 'state_2': 'category',
 'zip_2': 'category',
 'country_2': 'category',
 'url_2': 'int8',
 'phone_2': 'int8',
 'categories_2': 'object',
 'match': 'float16',
 'distance': 'float16',
 'name_1_fuzzy': 'float64',
 'address_1_fuzzy': 'float64',
 'categories_1_fuzzy': 'float64'}